<a href="https://colab.research.google.com/github/jenskuehne-cmd/Colab_Skripte_OEE/blob/main/Neue_Master_Tabelle_RM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dieses Notebook dient der Automatisierung der Datenverarbeitung von mehreren Google Sheets, die Benutzerrollen und -daten enthalten. Im Wesentlichen führt es die folgenden Schritte aus:

1.  **Authentifizierung bei Google Sheets**: Stellt eine Verbindung zu Google Sheets über gspread her.
2.  **Einlesen von Quelldaten**: Lädt Daten aus drei verschiedenen Google Sheets (`KAU`, `PTD`, `PS`) in Pandas DataFrames.
3.  **Analyse der Spaltenüberschriften**: Identifiziert gemeinsame Spaltenüberschriften in den drei Quell-Sheets.
4.  **Aktualisierung einer Master-Tabelle (`RoleMap`)**: Basierend auf der Spalte "No Aspire Role needed" in den `KAU`- und `PS`-Sheets werden entsprechende Einträge in der Master-Tabelle `RoleMap` aktualisiert. Wenn eine `UserID` in den Quell-Sheets den Wert "TRUE" in dieser Spalte hat, wird auch der Eintrag in der `RoleMap` auf "TRUE" gesetzt.
5.  **Extrahieren und Speichern relevanter Daten**: Filtert die `RoleMap`-Tabelle nach Benutzern, die den Wert "TRUE" in der Spalte "No Aspire Role needed" haben, und schreibt deren `UserID`, `Email` und den Flag-Wert in eine separate Tabelle namens `NoRolesNeeded`.

In [ ]:
# prompt: dieser Code führt 3 gSheet Tabellen zusammen. cih möchte erst einmal alle Überschriften der jeweiligen Tabellen aufgelistet haben
# # @title
# import gspread
# import pandas as pd
# from google.colab import auth
# from google.auth import default
# from datetime import datetime
# # Authentifizierung für Google Sheets
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)
# # Rollen-Spalten definieren
# ps_role_columns = ['Maintenance', 'Kalibrierung', 'MDG EAM', 'MDG MM', 'Analyse Reporting',
#                    'Ersatzteilmanagement', 'Engineering', 'Engineering - Capex']
# role_columns = ['Maintenance KAU Roles', 'Analyse Reporting', 'Ersatzteilmanagem.', 'Kalibrierung', 'EWM KAU', 'MDG EAM', 'MDG MM']
# # KAU DATEN
# spreadsheet_url_kau = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit?gid=2009282802#gid=2009282802'
# sheet_kau = gc.open_by_url(spreadsheet_url_kau)
# worksheet_kau = sheet_kau.get_worksheet(0)
# data_KAU = pd.DataFrame(worksheet_kau.get_all_records())
# # PTD DATEN
# spreadsheet_url_ptd = 'https://docs.google.com/spreadsheets/d/1wguV7OQ-0VlNLO5PTFucD4mPnNiCdwzRmpSZA33rL1I/edit?gid=1165668754#gid=1165668754'
# sheet_ptd = gc.open_by_url(spreadsheet_url_ptd)
# worksheet_ptd = sheet_ptd.worksheet("PTD User")
# data_ptd = pd.DataFrame(worksheet_ptd.get_all_records())
# # PS DATEN
# spreadsheet_url_ps = 'https://docs.google.com/spreadsheets/d/13ZOtTr2yKRPZnzsql8C8DOFaFzY8BXaculHtYliilhY/edit?pli=1&gid=734168569#gid=734168569'
# sheet_ps = gc.open_by_url(spreadsheet_url_ps)
# worksheet_ps = sheet_ps.worksheet("EndUserPS_Unique")
# data_ps = pd.DataFrame(worksheet_ps.get_all_records())
# # Funktion zur Zusammenführung von Rollen aus mehreren Spalten und Duplikate entfernen
# def combine_roles(row, role_columns):
#     roles = set()
#     for col in role_columns:
#         if col in row.index and isinstance(row[col], str):
#             roles_in_cell = row[col].split(',')
#             roles.update([r.strip() for r in roles_in_cell if r.strip().lower()

# @title
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from datetime import datetime

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# KAU DATEN
spreadsheet_url_kau = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit?gid=2009282802#gid=2009282802'
sheet_kau = gc.open_by_url(spreadsheet_url_kau)
worksheet_kau = sheet_kau.get_worksheet(0)
# Print KAU header
print("KAU Header:")
print(worksheet_kau.row_values(1)) # print the first row which contains the header

# PTD DATEN
spreadsheet_url_ptd = 'https://docs.google.com/spreadsheets/d/1wguV7OQ-0VlNLO5PTFucD4mPnNiCdwzRmpSZA33rL1I/edit?gid=1165668754#gid=1165668754'
sheet_ptd = gc.open_by_url(spreadsheet_url_ptd)
worksheet_ptd = sheet_ptd.worksheet("PTD User")
# Print PTD header
print("\nPTD Header:")
print(worksheet_ptd.row_values(1))

# PS DATEN
spreadsheet_url_ps = 'https://docs.google.com/spreadsheets/d/13ZOtTr2yKRPZnzsql8C8DOFaFzY8BXaculHtYliilhY/edit?pli=1&gid=734168569#gid=734168569'
sheet_ps = gc.open_by_url(spreadsheet_url_ps)
worksheet_ps = sheet_ps.worksheet("EndUserPS_Unique")
# Print PS header
print("\nPS Header:")
print(worksheet_ps.row_values(1))

# The rest of your code (dataframes, combine_roles function etc.) can remain the same
# ...

In [ ]:
# prompt: wie kann ihc den code kopierbar für excel erhalten

import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from datetime import datetime

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

def get_sheet_data(spreadsheet_url, worksheet_name):
    """
    Retrieves data from a Google Sheet and returns it as a Pandas DataFrame.

    Args:
        spreadsheet_url: The URL of the Google Sheet.
        worksheet_name: The name of the worksheet to retrieve data from.

    Returns:
        A Pandas DataFrame containing the sheet data, or None if an error occurs.
    """
    try:
        sheet = gc.open_by_url(spreadsheet_url)
        worksheet = sheet.worksheet(worksheet_name)
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])  # Exclude header row
        return df
    except Exception as e:
        print(f"Error retrieving data from sheet: {e}")
        return None

# Sheet URLs and worksheet names
sheet_urls = {
    "KAU": {
        "url": "https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit?gid=2009282802#gid=2009282802",
        "worksheet": "RM_KAU" # Replace with correct worksheet name if not "Sheet1"
    },
    "PTD": {
        "url": "https://docs.google.com/spreadsheets/d/1wguV7OQ-0VlNLO5PTFucD4mPnNiCdwzRmpSZA33rL1I/edit?gid=1165668754#gid=1165668754",
        "worksheet": "PTD User"
    },
    "PS": {
        "url": "https://docs.google.com/spreadsheets/d/13ZOtTr2yKRPZnzsql8C8DOFaFzY8BXaculHtYliilhY/edit?pli=1&gid=734168569#gid=734168569",
        "worksheet": "EndUserPS_Unique"
    }
}


# Get data for each sheet
dfs = {}
for sheet_name, sheet_info in sheet_urls.items():
    dfs[sheet_name] = get_sheet_data(sheet_info["url"], sheet_info["worksheet"])
    if dfs[sheet_name] is not None:
        print(f"\n{sheet_name} Header:")
        print(dfs[sheet_name].columns.tolist())  # Print column headers

# Now you have the dataframes (dfs["KAU"], dfs["PTD"], dfs["PS"])
# Process the data as needed
# ... your data manipulation code here ...


# Exporting DataFrames to CSV (optional, for easier use outside of Colab)
for sheet_name, df in dfs.items():
    if df is not None:
        df.to_csv(f"{sheet_name}_data.csv", index=False)  # Save to CSV files
        print(f"Saved {sheet_name} data to {sheet_name}_data.csv")

# Download the CSV files
from google.colab import files
for sheet_name, df in dfs.items():
    if df is not None:
        files.download(f"{sheet_name}_data.csv")

In [ ]:
# prompt: finder heraus welche Spaltenüberschriften in allen 3 Tabellen identisch sind

# Assuming dfs dictionary contains the dataframes as before

# Find common column headers
common_columns = set(dfs["KAU"].columns.tolist())
for df_name in ["PTD", "PS"]:
    common_columns = common_columns.intersection(set(dfs[df_name].columns.tolist()))

print("\nCommon column headers across all three sheets:")
print(list(common_columns))

In [ ]:
# prompt: # @title
# import gspread
# import pandas as pd
# from google.colab import auth
# from google.auth import default
# from datetime import datetime
# # Authentifizierung für Google Sheets
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)
# # KAU DATEN
# spreadsheet_url_kau = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit?gid=2009282802#gid=2009282802'
# sheet_kau = gc.open_by_url(spreadsheet_url_kau)
# worksheet_kau = sheet_kau.worksheet("RM_KAU")
# Ich möchte auf oben aufgeführttes source gsheet mit colab zugreifen und alle "UserID" die in spalte "No Aspire Role needed" ein "TRUE" haben in einer anderen target Tabelle auf true setzen
# Target Tabelle "https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit?gid=290247992#gid=290247992"
# im Reiter "RoleMap"
# in  Spalte "No Aspire Role needed" soll auf TRUE gesetzt werden wenn "UserID" aus source und target tabelle gliech sind und in source tabelle in Spalte"No Aspire Role needed" "TRUE" ist

# Assuming dfs dictionary contains the dataframes as before

# Target spreadsheet details
target_spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit?gid=290247992#gid=290247992'
target_worksheet_name = "RoleMap"

# Get the target worksheet
target_sheet = gc.open_by_url(target_spreadsheet_url)
target_worksheet = target_sheet.worksheet(target_worksheet_name)

# Get the data from the target worksheet
target_data = target_worksheet.get_all_values()
target_df = pd.DataFrame(target_data[1:], columns=target_data[0])


# Find UserIDs in KAU sheet with "No Aspire Role needed" as TRUE
kau_true_users = dfs["KAU"][dfs["KAU"]["No Aspire Role needed"] == "TRUE"]["UserID"].tolist()

# Update target sheet
for index, row in target_df.iterrows():
    if row["UserID"] in kau_true_users:
        target_df.loc[index, "No Aspire Role needed"] = "TRUE"

# Update the target worksheet with the modified data
target_worksheet.update([target_df.columns.values.tolist()] + target_df.values.tolist())

print("Target sheet updated successfully!")


In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Source Google Sheet (KAU Daten)
source_sheet_url = "https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit"
source_sheet = gc.open_by_url(source_sheet_url)
source_worksheet = source_sheet.worksheet("RM_KAU")

# Target Google Sheet (RoleMap)
target_sheet_url = "https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit"
target_sheet = gc.open_by_url(target_sheet_url)
target_worksheet = target_sheet.worksheet("RoleMap")

# Daten aus Source-Tabelle abrufen
source_data = source_worksheet.get_all_values()
source_df = pd.DataFrame(source_data[1:], columns=source_data[0])  # Erste Zeile als Header nutzen

# Daten aus Target-Tabelle abrufen
target_data = target_worksheet.get_all_values()
target_df = pd.DataFrame(target_data[2:], columns=target_data[1])  # Zweite Zeile als Header nutzen

# Sicherstellen, dass die relevanten Spalten existieren
source_column_userid = "UserID"
source_column_flag = "No Aspire Role needed"
target_column_flag = "No Aspire Role needed"

if source_column_userid not in source_df.columns or source_column_flag not in source_df.columns:
    raise ValueError("Spaltennamen in der Source-Tabelle sind nicht korrekt!")

if source_column_userid not in target_df.columns or target_column_flag not in target_df.columns:
    raise ValueError("Spaltennamen in der Target-Tabelle sind nicht korrekt!")

# Liste der UserIDs mit "TRUE" in "No Aspire Role needed"
true_user_ids = set(source_df[source_df[source_column_flag] == "TRUE"][source_column_userid])

# Spaltenindex in der Google Sheet Tabelle bestimmen
header_row = target_data[1]  # Zweite Zeile ist der Header
user_id_col_index = header_row.index(source_column_userid) + 1
flag_col_index = header_row.index(target_column_flag) + 1

# Updates als Batch vorbereiten (nur relevante Zellen)
updates = []
for i, row in enumerate(target_df.itertuples(), start=3):  # Startet ab Zeile 3 (wegen leerer erster Zeile)
    user_id = getattr(row, source_column_userid)
    if user_id in true_user_ids:
        cell_range = f"{chr(64 + flag_col_index)}{i}"  # Korrekt: Nur Zellreferenz, kein doppeltes Tabellenblatt
        updates.append({"range": cell_range, "values": [["TRUE"]]})

# Batch-Update durchführen (falls Änderungen vorhanden)
if updates:
    target_worksheet.batch_update(updates)
    print(f"✅ Update abgeschlossen! {len(updates)} Werte in 'No Aspire Role needed' aktualisiert.")
else:
    print("✅ Keine Änderungen notwendig.")


In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Source Google Sheet (KAU Daten)
source_sheet_url = "https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit"
source_sheet = gc.open_by_url(source_sheet_url)
source_worksheet = source_sheet.worksheet("RM_KAU")

# Target Google Sheet (RoleMap)
target_sheet_url = "https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit"
target_sheet = gc.open_by_url(target_sheet_url)
target_worksheet = target_sheet.worksheet("RoleMap")

# Daten aus Source-Tabelle abrufen
source_data = source_worksheet.get_all_values()
source_df = pd.DataFrame(source_data[1:], columns=source_data[0])  # Erste Zeile als Header nutzen

# Daten aus Target-Tabelle abrufen
target_data = target_worksheet.get_all_values()
target_df = pd.DataFrame(target_data[2:], columns=target_data[1])  # Zweite Zeile als Header nutzen

# Sicherstellen, dass die relevanten Spalten existieren
source_column_userid = "UserID"
source_column_flag = "No Aspire Role needed"
target_column_flag = "No Aspire Role needed"

if source_column_userid not in source_df.columns or source_column_flag not in source_df.columns:
    raise ValueError("Spaltennamen in der Source-Tabelle sind nicht korrekt!")

if source_column_userid not in target_df.columns or target_column_flag not in target_df.columns:
    raise ValueError("Spaltennamen in der Target-Tabelle sind nicht korrekt!")

# Liste der UserIDs mit "TRUE" in "No Aspire Role needed"
true_user_ids = set(source_df[source_df[source_column_flag] == "TRUE"][source_column_userid])

# Spaltenindex in der Google Sheet Tabelle bestimmen
header_row = target_data[1]  # Zweite Zeile ist der Header
user_id_col_index = header_row.index(source_column_userid) + 1
flag_col_index = header_row.index(target_column_flag) + 1

# Sicherstellen, dass der Spaltenindex korrekt ist
if flag_col_index < 1 or flag_col_index > 26:
    raise ValueError(f"Fehler: Der berechnete Spaltenindex {flag_col_index} ist ungültig!")

# **Richtige Spaltennummer für Pandas DataFrame finden**
flag_col_idx = target_df.columns.get_loc(target_column_flag)

# Updates als Batch vorbereiten (nur relevante Zellen)
updates = []
for i, row in enumerate(target_df.itertuples(index=False, name=None), start=3):  # Startet ab Zeile 3 (wegen leerer erster Zeile)
    user_id = row[0]  # **UserID ist in der ersten Spalte**
    current_value = row[flag_col_idx]  # **Hier wird die Checkbox-Spalte korrekt geholt**

    if user_id in true_user_ids and current_value != True:  # Nur ändern, wenn nötig
        cell_range = f"RoleMap!{chr(64 + flag_col_index)}{i}"  # Korrekt: Spalte als A-Z umrechnen
        updates.append({"range": cell_range, "values": [[True]]})  # ✅ **Echter boolescher Wert (True), keine Strings!**

# Batch-Update durchführen (falls Änderungen vorhanden)
if updates:
    target_worksheet.batch_update(updates)
    print(f"✅ Update abgeschlossen! {len(updates)} Checkboxen wurden auf 'true' gesetzt.")
else:
    print("✅ Keine Änderungen notwendig.")


PS no Aspire role übertragen in master Tabelle 6900

Bisher 691 aus KAU

PTD unbekannt

In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Neue Source Google Sheet (KAU Daten)
source_sheet_url = "https://docs.google.com/spreadsheets/d/13ZOtTr2yKRPZnzsql8C8DOFaFzY8BXaculHtYliilhY/edit"
source_sheet = gc.open_by_url(source_sheet_url)
source_worksheet = source_sheet.worksheet("EndUserPS_Unique")

# Target Google Sheet (RoleMap)
target_sheet_url = "https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit"
target_sheet = gc.open_by_url(target_sheet_url)
target_worksheet = target_sheet.worksheet("RoleMap")

# Daten aus Source-Tabelle abrufen
source_data = source_worksheet.get_all_values()
source_df = pd.DataFrame(source_data[1:], columns=source_data[0])  # Erste Zeile als Header nutzen

# Daten aus Target-Tabelle abrufen
target_data = target_worksheet.get_all_values()
target_df = pd.DataFrame(target_data[2:], columns=target_data[1])  # Zweite Zeile als Header nutzen

# Sicherstellen, dass die relevanten Spalten existieren
source_column_userid = "UserID"
source_column_flag = "No Aspire Role needed"
target_column_flag = "No Aspire Role needed"

if source_column_userid not in source_df.columns or source_column_flag not in source_df.columns:
    raise ValueError("Spaltennamen in der Source-Tabelle sind nicht korrekt!")

if source_column_userid not in target_df.columns or target_column_flag not in target_df.columns:
    raise ValueError("Spaltennamen in der Target-Tabelle sind nicht korrekt!")

# Liste der UserIDs mit "TRUE" in "No Aspire Role needed"
true_user_ids = set(source_df[source_df[source_column_flag] == "TRUE"][source_column_userid])

# Spaltenindex in der Google Sheet Tabelle bestimmen
header_row = target_data[1]  # Zweite Zeile ist der Header
user_id_col_index = header_row.index(source_column_userid) + 1
flag_col_index = header_row.index(target_column_flag) + 1

# Updates als Batch vorbereiten (nur relevante Zellen)
updates = []
for i, row in enumerate(target_df.itertuples(), start=3):  # Startet ab Zeile 3 (wegen leerer erster Zeile)
    user_id = getattr(row, source_column_userid)
    if user_id in true_user_ids:
        cell_range = f"{chr(64 + flag_col_index)}{i}"  # Korrekt: Nur Zellreferenz, kein doppeltes Tabellenblatt
        updates.append({"range": cell_range, "values": [["TRUE"]]})

# Batch-Update durchführen (falls Änderungen vorhanden)
if updates:
    target_worksheet.batch_update(updates)
    print(f"✅ Update abgeschlossen! {len(updates)} Werte in 'No Aspire Role needed' aktualisiert.")
else:
    print("✅ Keine Änderungen notwendig.")


In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Target Google Sheet
target_sheet_url = "https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit"
target_sheet = gc.open_by_url(target_sheet_url)
source_worksheet = target_sheet.worksheet("RoleMap")  # Quelle ist RoleMap
target_worksheet_no_roles = target_sheet.worksheet("NoRolesNeeded")  # Ziel ist NoRolesNeeded

# Daten aus Source-Tabelle abrufen
source_data = source_worksheet.get_all_values()
source_df = pd.DataFrame(source_data[2:], columns=source_data[1])  # Spaltenüberschriften in Zeile 2

# Sicherstellen, dass die relevanten Spalten existieren
source_column_userid = "UserID"
source_column_email = "Email"
source_column_flag = "No Aspire Role needed"

if any(col not in source_df.columns for col in [source_column_userid, source_column_email, source_column_flag]):
    raise ValueError("Spaltennamen in der Source-Tabelle sind nicht korrekt!")

# Daten für NoRolesNeeded filtern und schreiben
filtered_df = source_df[source_df[source_column_flag] == "TRUE"]
data_to_write = filtered_df[[source_column_userid, source_column_email, source_column_flag]]

target_worksheet_no_roles.clear()
target_worksheet_no_roles.update([data_to_write.columns.tolist()] + data_to_write.values.tolist())

print("✅ Daten erfolgreich in die Zieltabelle 'NoRolesNeeded' geschrieben.")
